## Mailing with Python and SMTPlib

In [1]:
import smtplib
import os

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

passkey = os.environ.get('auto_mailkey')
from_account = os.environ.get('auto_mailaccount')

#### simple email

In [ ]:
body = """Hello Email"""

#securely connect to server
smtp_server = smtplib.SMTP('smtp.gmail.com', 587)
smtp_server.ehlo() #initiate contact with server
smtp_server.starttls()

smtp_server.login(from_account, passkey)
smtp_server.sendmail(from_account, 'grantryan.king@gmail.com', body)
smtp_server.quit()

#### Multipart email MIME

In [ ]:
msg = MIMEMultipart()
msg['From'] = from_account
msg['To'] = 'grantryan.king@gmail.com'
msg['Subject'] = 'Subject Title'
body = """Multipart email"""

msg.attach(MIMEText(body, 'plain')) #'html' also optional

In [ ]:
#securely connect to server
smtp_server = smtplib.SMTP('smtp.gmail.com', 587)
smtp_server.ehlo() #initiate contact with server
smtp_server.starttls()

smtp_server.login(from_account, passkey)
text = msg.as_string()
smtp_server.sendmail(from_account, 'grantryan.king@gmail.com', text)
smtp_server.quit()

print('Email sent successfully')

#### Blind CC option

In [ ]:
msg = MIMEMultipart()
msg['From'] = from_account
msg['To'] = 'grantryan.king@gmail.com'
msg['Subject'] = 'Subject Title'
body = """Multipart email"""
#define bcc outside of multipart dict aceess to ensure blindness is respected
bcc = ['g.rantryanking@gmail.com', 'grantryanking@gmail.com']

msg.attach(MIMEText(body, 'plain')) #'html' also optional

#securely connect to server
smtp_server = smtplib.SMTP('smtp.gmail.com', 587)
smtp_server.ehlo() #initiate contact with server
smtp_server.starttls()

smtp_server.login(from_account, passkey)
text = msg.as_string()
smtp_server.sendmail(from_account, 'grantryan.king@gmail.com' + bcc, text)
smtp_server.quit()

print('Email sent successfully')

### Use context manager 


In [3]:
from email.message import EmailMessage

msg = EmailMessage()
msg['Subject'] = 'Another email'
msg['From'] = from_account
msg['To'] = 'grantryan.king@gmail.com'
msg.set_content('Look at what can be accomplished using smtplib WITH a context manager!')

with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
    smtp.ehlo() #initiate contact with server
    smtp.starttls()
    smtp.ehlo() #verify encrypted contact
    smtp.login(from_account, passkey)
    smtp.send_message(msg)

#### use local debug server to test sending email

In [ ]:
#in your terminal, type >> python -m smtpd -c DebuggingServer -n localhost:1025
#this will start a test email server that prints incoming emails to the console and discards them

with smtplib.SMTP('localhost', 1025) as smtp:
    subject = 'Look at this email'
    body = 'Look at what can be accomplished using smtplib WITH a context manager!'
    
    msg = f'{subject}\n\n{body}'
    
    smtp.sendmail(from_account, 'grantryan.king@gmail.com', msg)

### attach things

In [4]:
from email.message import EmailMessage
import imghdr

msg = EmailMessage()
msg['Subject'] = 'Another email'
msg['From'] = from_account
msg['To'] = 'grantryan.king@gmail.com'
msg.set_content('Look at what can be attached using smtplib!')

with open('pot_collage.jpg', 'rb') as file:
    file_data = file.read()
    #figure out image type with filename
    file_type = imghdr.what(file.name)
    file_name = file.name
    
msg.add_attachment(file_data, maintype='image', subtype=file_type, filename=file_name)

with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
    smtp.ehlo() #initiate contact with server
    smtp.starttls()
    smtp.ehlo() #verify encrypted contact
    smtp.login(from_account, passkey)
    smtp.send_message(msg)

#### attach multiple things

In [6]:
from email.message import EmailMessage
import imghdr

msg = EmailMessage()
msg['Subject'] = 'Another email'
msg['From'] = from_account
msg['To'] = 'grantryan.king@gmail.com'
msg.set_content('Look at how many can be attached using smtplib!')

files = ['pot_collage.jpg', 'pot_collage2.jpg']
for file in files:
    with open(file, 'rb') as file:
        file_data = file.read()
        #figure out image type with filename
        file_type = imghdr.what(file.name)
        file_name = file.name
    
    msg.add_attachment(file_data, maintype='image', subtype=file_type, filename=file_name)

with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
    smtp.ehlo() #initiate contact with server
    smtp.starttls()
    smtp.ehlo() #verify encrypted contact
    smtp.login(from_account, passkey)
    smtp.send_message(msg)

#### send other filetypes
application maintype and octet-stream subtype is for generic data

In [7]:
from email.message import EmailMessage
import imghdr

msg = EmailMessage()
msg['Subject'] = 'Another email'
msg['From'] = from_account
msg['To'] = 'grantryan.king@gmail.com'
msg.set_content('Look at how many can be attached using smtplib!')

files = ['ERD.pdf']
for file in files:
    with open(file, 'rb') as file:
        file_data = file.read()
        file_name = file.name
    
    msg.add_attachment(file_data, maintype='application', subtype='octet-stream', filename=file_name)

with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
    smtp.ehlo() #initiate contact with server
    smtp.starttls()
    smtp.ehlo() #verify encrypted contact
    smtp.login(from_account, passkey)
    smtp.send_message(msg)

### Send HTML email
with plaintext as fallback for those who have HTML emails turned off